In [ ]:
#!pip install -U transformers --quiet

In [ ]:
#!pip install -q transformers datasets scikit-learn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

DATA_DIR = "/content/drive/MyDrive/archive/data"

with open(f"{DATA_DIR}/emotions.txt", "r") as f:
    emotions = [line.strip() for line in f.readlines()]

columns = ["text", "labels", "id"]
train_df = pd.read_csv(f"{DATA_DIR}/train.tsv", sep="\t", names=columns)
test_df = pd.read_csv(f"{DATA_DIR}/test.tsv", sep="\t", names=columns)
dev_df = pd.read_csv(f"{DATA_DIR}/dev.tsv", sep="\t", names=columns)

df = pd.concat([train_df, test_df, dev_df])

In [ ]:
emotion_to_mood = {
    "amusement": "happy", "approval": "happy", "caring": "calm", "confusion": "neutral", "curiosity": "neutral",
    "desire": "neutral", "excitement": "happy", "gratitude": "happy", "joy": "happy", "love": "happy",
    "optimism": "happy", "pride": "happy", "relief": "happy", "admiration": "happy",
    "anger": "angry", "annoyance": "angry", "disappointment": "sad", "disapproval": "angry",
    "embarrassment": "sad", "fear": "fearful", "grief": "sad", "nervousness": "fearful", "remorse": "sad",
    "sadness": "sad", "realization": "neutral", "surprise": "happy", "neutral": "neutral"
}

def get_first_mood(label_str):
    try:
        label_indices = list(map(int, label_str.split(',')))
        emotion = emotions[label_indices[0]]
        return emotion_to_mood.get(emotion, None)
    except:
        return None

df['mood'] = df['labels'].apply(get_first_mood)
df = df[df['mood'].notnull()].reset_index(drop=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['label'] = le.fit_transform(df['mood'])

label_map = dict(zip(le.classes_, le.transform(le.classes_)))
print(label_map)

{'angry': np.int64(0), 'calm': np.int64(1), 'fearful': np.int64(2), 'happy': np.int64(3), 'neutral': np.int64(4), 'sad': np.int64(5)}


In [ ]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    stratify=df['label'],
    random_state=42
)

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
import torch

class MoodDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        return {
            key: torch.tensor(val[idx]) for key, val in self.encodings.items()
        } | {'labels': torch.tensor(self.labels[idx])}
    def __len__(self):
        return len(self.labels)

train_dataset = MoodDataset(train_encodings, train_labels)
val_dataset = MoodDataset(val_encodings, val_labels)

In [ ]:
#!pip install -U transformers datasets

In [ ]:
#!pip install -U transformers datasets --quiet

In [ ]:
import transformers
print(transformers.__version__)

4.52.4


In [ ]:
from huggingface_hub import login
login()

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score


le = LabelEncoder()
df['label'] = le.fit_transform(df['mood'])

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=len(le.classes_)
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc
    }

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-14-3358485684.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: manasam (manasam-cybic-tech) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.738500,0.735408,0.716581
2,0.505300,0.759015,0.729005
3,0.239000,1.010781,0.713031


TrainOutput(global_step=8031, training_loss=0.5542132651697519, metrics={'train_runtime': 2991.1967, 'train_samples_per_second': 42.946, 'train_steps_per_second': 2.685, 'total_flos': 8450115012679680.0, 'train_loss': 0.5542132651697519, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.7590146660804749,
 'eval_accuracy': 0.7290051377860812,
 'eval_runtime': 36.7653,
 'eval_samples_per_second': 291.171,
 'eval_steps_per_second': 4.57,
 'epoch': 3.0}

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(df["mood"])

def predict_mood(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128).to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        pred_class = torch.argmax(probs, dim=1).item()
    return label_encoder.inverse_transform([pred_class])[0]

In [ ]:
print(predict_mood("I feel enthusiastic"))

happy


In [ ]:
# Save the fine-tuned model
model_path = "./mood_classifier"
trainer.save_model(model_path)  # Saves the model and tokenizer
tokenizer.save_pretrained(model_path)

# Also save the LabelEncoder (you'll need it to decode predictions later)
import joblib
joblib.dump(le, f"{model_path}/label_encoder.pkl")

['./mood_classifier/label_encoder.pkl']

In [ ]:
!ls /content/drive/MyDrive/archive

analyze_data.py       GoEmotionsFormat.PNG	 README.md
calculate_metrics.py  goemotions_model_card.pdf  replace_emotions.py
data		      mood_classifier		 tables
extract_words.py      plots


In [ ]:
model_path = "/content/drive/MyDrive/archive/mood_classifier"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

import joblib
joblib.dump(le, f"{model_path}/label_encoder.pkl")

['/content/drive/MyDrive/archive/mood_classifier/label_encoder.pkl']